# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'C:\Users\nikita.saprykin\Desktop\RCS\урок_4\retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

item_features = pd.read_csv(r'C:\Users\nikita.saprykin\Desktop\RCS\урок_4\product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [4]:
# Класс ниже - перенесен в отдельный модуль.

# # Для работы с матрицами
# from scipy.sparse import csr_matrix

# # Матричная факторизация
# from implicit.als import AlternatingLeastSquares
# from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
# from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class RENAMED_FOR_INFO_ONLY_MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        # your_code
        
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
       
        # необходимый тип матрицы для implicit
        user_item_matrix = user_item_matrix.astype(float) 

        # переведем в формат saprse matrix
        # sparse_user_item = csr_matrix(user_item_matrix).tocsr()
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,
                                             calculate_training_loss=True,
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
        
        return model
    
    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
       
        recs = self.model.recommend(userid=self.userid_to_id[user], 
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),   
                                             # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)
        
        res = [self.id_to_itemid[rec[0]] for rec in recs]
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code 
        userlist = self.model.similar_users(self.userid_to_id[user], N=N+1)[1:N+1]
        user_id_list = [self.id_to_userid[x[0]] for x in userlist]
        res = [self.get_similar_items_recommendation(user_id, N=1)[0] for user_id in user_id_list]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

Проверка, что все работает

In [5]:
import sys
sys.path.append("..")

from src.metrics import precision_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender



In [6]:
tester = MainRecommender(data)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/89051 [00:00<?, ?it/s]

In [7]:
tester.get_similar_items_recommendation(1234)

[5978656, 955031, 1107749, 1074612, 962568]

In [8]:
tester.get_similar_users_recommendation(1234)

[981760, 963365, 1082185, 5978656, 1127831]